In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# ! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## Read Data

In [2]:
df = pd.read_csv('./amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)


C:\Users\Ayan\AppData\Local\Temp\ipykernel_12620\3693539848.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('./amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)
C:\Users\Ayan\AppData\Local\Temp\ipykernel_12620\3693539848.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('./amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)
C:\Users\Ayan\AppData\Local\Temp\ipykernel_12620\3693539848.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)


## Keep Reviews and Ratings

In [20]:
df = df[['star_rating', 'review_body']]

# let ratings with the values of 1 and 2 form class 1, ratings with the value of 3 form class 2, and ratings with the values of 4 and 5 form class 3

In [21]:
class_one = df[(df['star_rating']==1) | (df['star_rating']==2)]
class_two = df[df['star_rating']==3]
class_three = df[(df['star_rating']==4) | (df['star_rating']==5)]
print(len(class_one), len(class_two), len(class_three))

588295 324095 3198869


 ## We form three classes and select 20000 reviews randomly from each class.



In [45]:
class_one = class_one.sample(n=20000, random_state=100)
class_two = class_two.sample(n=20000, random_state=100)
class_three = class_three.sample(n=20000, random_state=100)
dataset = pd.concat([class_one, class_two, class_three])
dataset.reset_index(drop=True)
train = dataset.sample(frac=0.8, random_state=100)
test = dataset.drop(train.index)
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)
print(len(train), len(test))

48000 12000


# Data Cleaning



# convert all reviews into lowercase

In [65]:
avg_len_before = (train['review_body'].str.len().sum() + test['review_body'].str.len().sum())/60000

#Covert all reviews to lower case
train['review_body'] = train['review_body'].str.lower()
test['review_body'] = test['review_body'].str.lower()

# remove the HTML and URLs from the reviews

In [74]:
# URL Remover code (Ref:https://stackoverflow.com/questions/51994254/removing-url-from-a-column-in-pandas-dataframe)
train['review_body'] = train['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
test['review_body'] = test['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

def html_tag_remover(review):
    soup = BeautifulSoup(review, 'html.parser')
    review = soup.get_text()
    return review  

train['review_body'] = train['review_body'].apply(lambda review: html_tag_remover(review))
test['review_body'] = test['review_body'].apply(lambda review: html_tag_remover(review))

   star_rating                                        review_body
0            3                                              so so
1            1  i haven't noticed any improvement. i've been u...
2            2  very small amount of makeup in tube.  won't bu...
3            1  this razor looks good but it is a piece of jun...
4            1  my own fault- i read reviews for an hour but n...
5            4  this baby lasts me about 10 shaves per charge ...
6            3  bottle arrived open and leaking! this is ok bu...
7            3  i saw this on a talk show, and it was so hyped...
8          5.0  this product is wonderful.  a beauty and suppl...
9            5  my significant other uses it to moisturizer hi...
10           5                                         very good!
11           3  i've been using this for about a month now, an...
12           3  when placing order you have a choice of shade ...
13           1  i have had these for a bit now and i still can...
14        

# remove non-alphabetical characters

# remove extra spaces

# perform contractions on the reviews

# Pre-processing

## remove the stop words 

In [ ]:
from nltk.corpus import stopwords
 

## perform lemmatization  

In [ ]:
from nltk.stem import WordNetLemmatizer


# TF-IDF Feature Extraction

# Perceptron

# SVM

# Logistic Regression

# Naive Bayes